### 연속체인을 이용한 의도분류 예제

In [12]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    model="gpt-3.5-turbo-0125", 
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)


In [4]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "안녕, 내 이름은 {name}!"),
    ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?")
])


prompt = template.format_messages(
    language="Korean",
    name="랭체인 BOT",
    country_a="South Korea",
    country_b="France"
)


# template.format_messages(
#     language="Korean",
#     name="랭체인 BOT",
#     country_a="South Korea",
#     country_b="France"
# )



chat.predict_messages(prompt)

/Users/dc/test_Langchain/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict_messages` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


AIMessage(content='한국과 프랑스 사이의 거리는 대략 8,500km입니다. 제 이름은 랭체인 BOT이에요.', response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 60, 'total_tokens': 98}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fe396711-1bfe-4c44-9871-a9e83105557f-0')

In [6]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):

    def parse(self, text):
        items =  text.strip().split(",")
        return list(map(str.strip, items))
    
p = CommaOutputParser()

p.parse("Hello, how, are, you")

['Hello', 'how', 'are', 'you']

In [11]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase.Do NOT reply with anything else."),
    ("human", "{question}"),
])


chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items": 5,
    "question": "What are the WolrdCup winners after 1980?"
})

# prompt = template.format_messages(
#     max_items=10,
#     question="What are the colors?"
# )

# result = chat.predict_messages(prompt)

# p = CommaOutputParser()

# p.parse(result.content)

['italy', 'argentina', 'germany', 'france', 'brazil']

In [37]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler


chat = ChatOpenAI(
    model="gpt-3.5-turbo-0125", 
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)


# intent_prompt = ChatPromptTemplate.from_messages([
#     ("system", "당신은 질문에 대한 의도를 정확하게 파악하여 분류하는 기능을 가진 시스템입니다. 질문이 오면 해당 질문의 맥락을 파악하여 그 의도를 대표할 수 있는 한 개의 단어로 대답해주세요."),
#     ("human", "{question}"),
# ])

intent_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 질문에 대한 의도를 정확하게 파악하여 분류한 후 후속 프로세스를 진행하는 Agent입니다. 질문이 오면 해당 질문의 맥락을 파악하고 당신이 Agent라는 부분을 고려하여 그 의도를 대표할 수 있는 한 개의 단어로 대답해주세요."),
    ("human", "{question}"),
])

intent_chain = intent_prompt | chat


# small_intent_prompt = ChatPromptTemplate.from_messages([
#     ("system", "당신은 단어의 성질을 정확하게 파악하여 카테고리를 세분화 하는 기능을 가진 시스템입니다. 단어가 주어지면 성질을 파악하여 세분화된 카테고리 정보를 리스트로 반환해주세요. 카테고리 정보는 5개 정도면 충분합니다."),
#     ("human", "{intent}"),
# ])

small_intent_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 단어에 대한 의도를 정확하게 파악할 수 있는 전문가입니다. 단어가 주어지면 이와 관련된 의도를 추론하여 관련 단어 5개를 생성해주세요."),
    ("human", "{intent}"),
])

small_intent_chain = small_intent_prompt | chat 


final_chain = {"intent": intent_chain} | small_intent_chain

final_chain.invoke({
    "question": "영화 타이타닉 틀어줘."
})


영화1. 배우
2. 감독
3. 스토리
4. 촬영
5. 흥행

AIMessage(content='1. 배우\n2. 감독\n3. 스토리\n4. 촬영\n5. 흥행', response_metadata={'finish_reason': 'stop'}, id='run-f74e50f6-ab00-4016-8876-da3077c9eac2-0')